<a href="https://colab.research.google.com/github/iuri/regressionml/blob/main/Canny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob, os, time 
from google.colab import drive

import cv2
import matplotlib.pyplot as plt
import datetime
import os
import numpy as np
import time 


In [3]:
###
# utils.py
###
def plot_image(image, figsize = 10, cvtColorParam=None):
    plt.figure(figsize = (figsize, figsize))

    # plt.subplot(figsize[0], figsize[1], i+1)
    if cvtColorParam != None:
        image = cv2.cvtColor(image, cvtColorParam)

    plt.imshow(image)    #If the image is grayscale, as in our case, then we will reshape the output in the following way.
                                                                        #Also, we set the coloring to grayscale so that it doesn't look like it came out of an infrared camera :)
    # else:
    #     plt.imshow(image.reshape((img_rows, img_cols, 1)))
    plt.axis('off')

    # plt.tight_layout()   #Tight layout so that all of the generated images form a nice grid
        
    plt.show()

def get_image_from_webcam():
    # Webcam
    cap = cv2.VideoCapture(0)
    success, img = cap.read()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cap.release()

    return img
    
def save_image_from_webcam(folder):
    # Webcam
    img = get_image_from_webcam()
    name = f"{folder}/{datetime.datetime.now()}.png"
    cv2.imwrite(name , img)

    return name

def save_image(cv2image, filename=f"assets/{datetime.datetime.now()}.png"):
    sucess = cv2.imwrite(filename , cv2image)
    print(f'Imagem salva em: {filename}')
    return sucess


#Função para retornar tamanho da amostra e passo nos frames
def amostra(n_frames, e):
    import math
    import scipy.stats as st    
    
    Z = st.norm.ppf(1 - e/2) #Z-escore
    P = 0.50 #Desvio padrão 
    C = (Z**2)*P*(1-P)/(e**2) #Constante
    amos = math.ceil(C/(1 + C/n_frames))
    step = math.floor(n_frames/amos)
    return amos, step

def extract_images_from_video(file, error_rate = 0.1):
    import cv2

    video = cv2.VideoCapture(file)
    n_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    e = float(error_rate)
    
    amos, step = amostra(n_frames, e)
    
    images = []
    success, frame = video.read()
    start = 0

    while success and start + step <= n_frames:   
        images.append(frame)
        save_image(frame, f'assets/dataset/{start + step}.png')

        #Extrai frames específicos a cada passo
        video.set(cv2.CAP_PROP_POS_FRAMES, (start + step))
        success, frame = video.read()

        start += step

    return images

def get_images_from_folder(folder, cutArray = []):
    # cut=(50:600, 330:950)
    # import glob

    filenames = sorted(glob.glob(f'{folder}/*.png'))    
#    filenames = sorted(glob.glob(f'{folder}/*.png'), key=os.path.getmtime)
    images = []

    for file in filenames:
        img = cv2.imread(file)
        if len(cutArray) > 0:
            images.append(
                img[
                    cutArray[0] : cutArray[1], 
                    cutArray[2] : cutArray[3]
                ]
            )

        else:
            images.append(img)

    return images, filenames

def label_on_image(image, text, color = (0,0,255)):
    i = image.copy()
    cv2.putText(i, text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    return i 

def auto_canny(image, sigma=0.33):
    import argparse
	
    # compute the median of the single channel pixel intensities
    v = np.median(image)
	# apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)
    
    # return the edged image
    return edged

def play_images(all_images, rows = 1, columns = 1, sec = 60, time_between_frames = 0.2):
    height = all_images[0][0].shape[0]*rows
    width = all_images[0][0].shape[1]*columns

    n_frames = len(all_images[0])
    fps = n_frames // sec

    print(f'FPS: {fps}', f'height: {height}',f'width: {width}')

    for i in range(n_frames):
        if rows == 1:
            if columns == 1:
                im2show = np.hstack([all_images[0][i]])

            elif columns == 2:
                im2show = np.hstack([all_images[0][i], all_images[1][i]])
            
            elif columns == 3:
                im2show = np.hstack([all_images[0][i], all_images[1][i], all_images[2][i]])
            
            elif columns == 4:
                im2show = np.hstack([all_images[0][i], all_images[1][i], all_images[2][i], all_images[3][i]])

        elif rows == 2:
            if columns == 1:
                im2show = np.vstack([
                    np.hstack([all_images[0][i]]),
                    np.hstack([all_images[1][i]]),
                ])

            elif columns == 2:
                im2show = np.vstack([
                    np.hstack([all_images[0][i], all_images[1][i]]),
                    np.hstack([all_images[2][i], all_images[3][i]])
                ])

            elif columns == 3:
                im2show = np.vstack([
                    np.hstack([all_images[0][i], all_images[1][i], all_images[2][i]]),
                    np.hstack([all_images[3][i], all_images[4][i], all_images[5][i]])
                ])

            elif columns == 4:
                im2show = np.vstack([
                    np.hstack([all_images[0][i], all_images[1][i], all_images[2][i], all_images[3][i]]),
                    np.hstack([all_images[4][i], all_images[5][i], all_images[6][i], all_images[7][i]])
                ])
                
        cv2.imshow("image", im2show)
        k = cv2.waitKey(30) & 0xff
        if k == 27: 
            break

        time.sleep(time_between_frames)

    print(f'Stopped at frame {i}')
    cv2.destroyAllWindows() 

        

In [4]:
# to mount Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
print(os.listdir("/content/gdrive/MyDrive/BKP/MESTRADO/dataset/temp"))

['Cópia de 3761.png', 'Cópia de 3755.png', 'Cópia de 3758.png', 'Cópia de 3757.png', 'Cópia de 3760.png', 'Cópia de 3756.png', 'Cópia de 3753.png', 'Cópia de 3754.png', 'Cópia de 3762.png', 'Cópia de 3759.png']


In [13]:
folder = "/content/gdrive/MyDrive/BKP/MESTRADO/dataset/temp"

In [15]:
# images, files = get_images_from_folder(folder, cutArray=[50,600,330,950])
cutArray = []
filenames = sorted(glob.glob(f'{folder}/*.png'))    
images = []
for file in filenames:
    img = cv2.imread(file)
    if len(cutArray) > 0:
        images.append(
            img[
                cutArray[0] : cutArray[1], 
                cutArray[2] : cutArray[3]
            ]
        )

    else:
        images.append(img)

In [16]:
f'Foram carregadas {len(images)} imagens'

'Foram carregadas 58 imagens'

In [17]:
sec = 60
rows = 2
columns = 3
height = images[0].shape[0]*rows
width = images[0].shape[1]*columns
n_frames = len(images)
fps = n_frames // sec
f'FPS: {fps}', f'height: {height}',f'width: {width}'

('FPS: 0', 'height: 1920', 'width: 3840')

In [18]:
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
video = cv2.VideoWriter("outputs/canny.mp4", fourcc, float(fps), (width, height))
 
a = 75
b = 175
    
for image in images:
   if image is not None:
     #print("yes")
     if image.any():
       if image != 'None':
         frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
         frame = cv2.GaussianBlur(frame,(7,7), 0)
         # Canny from image
         suavizacao = cv2.GaussianBlur(frame,(7,7), 0)
         canny_from_frame = cv2.Canny(suavizacao, a, b)#, L2gradient = True)
         # canny_from_frame = cv2.bitwise_and(frame, frame, mask=canny_from_frame)
         canny_from_frame = cv2.cvtColor(canny_from_frame, cv2.COLOR_GRAY2BGR)
         # threshold
         thresh = cv2.threshold(frame, 60, 255, cv2.THRESH_BINARY)[1]
         # morphology edgeout = dilated_mask - mask
         # morphology dilate
         kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
         dilate = cv2.morphologyEx(thresh, cv2.MORPH_DILATE, kernel)
         # get absolute difference between dilate and thresh
         diff = cv2.absdiff(dilate, thresh)
         # invert
         edges = 255 - diff
         edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

         ret, img_otsu = cv2.threshold(suavizacao, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
         canny_from_otsu = cv2.Canny(img_otsu, a, b)
         canny_from_otsu = cv2.cvtColor(canny_from_otsu, cv2.COLOR_GRAY2BGR)




         autoCannyOtsu = auto_canny(img_otsu)
         autoCannyOtsu = cv2.cvtColor(autoCannyOtsu, cv2.COLOR_GRAY2BGR)
          
         autoCanny = auto_canny(frame)
         autoCanny = cv2.cvtColor(autoCanny, cv2.COLOR_GRAY2BGR)
          
         img_otsu = cv2.bitwise_not(img_otsu)
         img_otsu = cv2.cvtColor(img_otsu, cv2.COLOR_GRAY2BGR)


         # Labels on images
         image = label_on_image(image, "Real")
         img_otsu = label_on_image(img_otsu, "Threshold OTSU")
         canny_from_frame = label_on_image(canny_from_frame, "Canny")
         # autoCanny2 = label_on_image(autoCanny2, "Auto canny 0.66")
         autoCanny = label_on_image(autoCanny, "Auto canny")
         edges = label_on_image(edges, "edges")
         canny_from_otsu = label_on_image(canny_from_otsu, "Canny from OTSU")
         autoCannyOtsu = label_on_image(autoCannyOtsu, "Autocanny from otsu")
         im2show = np.vstack([
             np.hstack([image, canny_from_frame, autoCanny]),
             np.hstack([img_otsu, canny_from_otsu, autoCannyOtsu])
             # np.hstack([canny_from_frame, sobel, laplacian])
         ])

         cv2.imshow("image", im2show)

         video.write(im2show)

         k = cv2.waitKey(30) & 0xff
         if k == 27: 
           break
         time.sleep(0.2)

cv2.destroyAllWindows() 
    
video.release()

<ipython-input-18-9f41765e5b93>:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if image != 'None':


DisabledFunctionError: ignored